In [1]:
import meb
from meb import utils
from meb import datasets
from meb import core
from meb import models

from functools import partial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import timm



pd.set_option("display.max_columns", 50)
%load_ext autoreload
%autoreload 2

In [35]:
c = datasets.CrossDataset(resize=64, optical_flow=True)
df = c.data_frame
data = c.data

In [12]:
class Config(core.Config):
    device = torch.device("cuda:0")
    num_workers = 0
    evaluation_fn = [
        partial(utils.MultiLabelF1Score, average="macro"),
        partial(utils.MultiLabelF1Score, average="binary")
    ]
    epochs = 200
    batch_size = 128
    model = partial(meb.models.SSSNet, num_classes=len(core.Config.action_units))

In [5]:
from tqdm import tqdm

In [6]:
# Only validate casme2
class IValidator(core.CrossDatasetValidator):
    def __init__(self, config: "Config"):
        super().__init__(config)
        
    def validate_n_times(
        self, df: pd.DataFrame, input_data, n_times: int = 5
    ) -> None:
        self.verbose = False
        self.disable_tqdm = True
        au_results = []
        dataset_results = []
        casme2_idx = df["dataset"] == "casme2"
        for n in tqdm(range(n_times)):
            outputs_list = self.validate(df, input_data, seed_n=n + 45)
            au_result, dataset_result = self.printer.results_to_list(outputs_list, df[casme2_idx])
            au_results.append(au_result)
            dataset_results.append(dataset_result)

        aus = [i for i in self.cf.action_units]
        dataset_names = df["dataset"].unique().tolist()
        aus.append("Average")
        dataset_names.append("Average")
        au_results = np.array(au_results)
        dataset_results = np.array(dataset_results)
        for i in range(len(self.cf.evaluation_fn)):
            if len(self.cf.evaluation_fn) > 1:
                print(self.printer.metric_name(self.cf.evaluation_fn[i]))
            au_result = self.printer.list_to_latex(list(au_results[:, i].mean(axis=0)))
            dataset_result = self.printer.list_to_latex(
                list(dataset_results[:, i].mean(axis=0))
            )
            print("AUS:", aus)
            print(au_result)
            print("\nDatasets: ", dataset_names)
            print(dataset_result)
    
    def validate(self, df: pd.DataFrame, input_data: np.ndarray, seed_n: int = 1):
        utils.set_random_seeds(seed_n)
        dataset_names = df["dataset"].unique()
        # Create a boolean array with the AUs
        labels = np.array(df[self.cf.action_units])
        outputs_list = []
        for dataset_name in dataset_names:
            if dataset_name != "casme2":
                continue
            train_metrics, test_metrics, outputs_test = self.validate_split(
                df, input_data, labels, dataset_name
            )
            outputs_list.append(outputs_test)
            if self.verbose:
                self.printer.print_train_test_evaluation(
                    train_metrics, test_metrics, dataset_name, outputs_test.shape[0]
                )

        # Calculate total f1-scores
        predictions = torch.cat(outputs_list)
        idx = df["dataset"] == "casme2"
        metrics = self.evaluation_fn(labels[idx], predictions)
        if self.verbose:
            self.printer.print_test_validation(metrics)
        return outputs_list

In [7]:
#how is casme3 of extracted for samples with onset == apex?

## First row
concatenating other datasets, no casme3

In [13]:
use_datasets = ["casme2"]
for dataset in ["casme", "samm", "mmew", "fourd"]:
    use_datasets.append(dataset)
    idx = df["dataset"].isin(use_datasets)
    print(use_datasets)
    IValidator(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=5)

['casme2', 'casme']


  0%|                                                     | 0/5 [00:00<?, ?it/s]/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
 20%|█████████                                    | 1/5 [00:05<00:21,  5.43s/it]/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
 40%|██████████████████                           | 2/5 [00:10<00:16,  5.49s/it]/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
90.7 & 79.2 & 94.2 & 49.8 & 48.7 & 45.9 & 55.8 & 48.4 & 56.3 & 64.0 & 71.0 & 76.5 & 65.1

Datasets:  ['casme', 'casme2', 'Average']
65.1 & 65.1
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
83.7 & 62.9 & 94.3 & 0.0 & 0.0 & 0.0 & 14.4 & 0.0 & 19.2 & 36.1 & 44.4 & 56.3 & 34.3

Datasets:  ['casme', 'casme2', 'Average']
34.3 & 34.3
['casme2', 'casme', 'samm']


100%|█████████████████████████████████████████████| 5/5 [00:44<00:00,  8.95s/it]


MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
95.5 & 84.7 & 94.4 & 49.8 & 48.7 & 63.5 & 63.8 & 48.2 & 71.3 & 68.9 & 70.3 & 77.9 & 69.7

Datasets:  ['casme', 'casme2', 'samm', 'Average']
69.7 & 69.7
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
91.9 & 72.5 & 94.3 & 0.0 & 0.0 & 34.0 & 30.1 & 0.0 & 48.4 & 43.0 & 42.9 & 58.9 & 43.0

Datasets:  ['casme', 'casme2', 'samm', 'Average']
43.0 & 43.0
['casme2', 'casme', 'samm', 'mmew']


100%|█████████████████████████████████████████████| 5/5 [01:26<00:00, 17.39s/it]


MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
93.7 & 82.8 & 96.2 & 48.1 & 52.9 & 68.0 & 56.3 & 51.0 & 69.1 & 67.3 & 61.2 & 80.3 & 68.9

Datasets:  ['casme', 'casme2', 'samm', 'mmew', 'Average']
68.9 & 68.9
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
88.8 & 69.4 & 96.3 & 0.0 & 8.4 & 43.8 & 15.3 & 5.8 & 44.6 & 39.6 & 25.2 & 63.4 & 41.7

Datasets:  ['casme', 'casme2', 'samm', 'mmew', 'Average']
41.7 & 41.7
['casme2', 'casme', 'samm', 'mmew', 'fourd']


100%|█████████████████████████████████████████████| 5/5 [01:57<00:00, 23.45s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
93.1 & 84.2 & 96.5 & 49.8 & 63.8 & 72.6 & 54.0 & 53.4 & 75.6 & 66.8 & 69.6 & 79.7 & 71.6

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'Average']
71.6 & 71.6
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
87.8 & 71.9 & 96.4 & 0.0 & 30.2 & 54.0 & 10.7 & 10.3 & 57.2 & 38.4 & 41.5 & 62.3 & 46.7

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'Average']
46.7 & 46.7


In [8]:
idx = df["dataset"].isin(["casme2", "casme"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [01:43<00:00, 10.35s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.89970545 0.78942949 0.94140056 0.49803922 0.48697395 0.45877378
 0.5599481  0.48387097 0.53712028 0.64364151 0.71085752 0.78906151]

Overall: 
0.6499018604891913


In [9]:
idx = df["dataset"].isin(["casme2", "casme", "samm"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [03:05<00:00, 18.55s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.940153   0.84953509 0.94567996 0.49784198 0.48697395 0.66927513
 0.63513766 0.48735037 0.71137314 0.69943936 0.71758283 0.78397535]

Overall: 
0.7020264849231566


In [10]:
idx = df["dataset"].isin(["casme2", "casme", "samm", "mmew"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [05:49<00:00, 34.93s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.94136162 0.82865244 0.96406015 0.48695131 0.52133649 0.6849995
 0.55342873 0.5267194  0.73422655 0.7056407  0.64254987 0.80775454]

Overall: 
0.6998067749403412


In [11]:
idx = df["dataset"].isin(["casme2", "casme", "samm", "mmew", "fourd"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [07:57<00:00, 47.77s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.93350339 0.8393631  0.96288537 0.49803922 0.61912694 0.73321722
 0.54682375 0.53443039 0.75882267 0.67309841 0.71125812 0.78507218]

Overall: 
0.7163033960611523


## Second row
Adding casme3 to the concatenated datasets

In [32]:
use_datasets = ["casme2"]
for dataset in ["casme3a", "casme", "samm", "mmew", "fourd"]:
    use_datasets.append(dataset)
    idx = df["dataset"].isin(use_datasets)
    print(use_datasets)
    IValidator(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=5)

['casme2', 'casme3a']


100%|█████████████████████████████████████████████| 5/5 [01:46<00:00, 21.20s/it]


MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
93.0 & 85.6 & 92.9 & 49.4 & 48.7 & 51.3 & 49.5 & 48.4 & 55.3 & 74.5 & 48.4 & 76.9 & 64.5

Datasets:  ['casme2', 'casme3a', 'Average']
64.5 & 64.5
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
87.7 & 74.0 & 92.8 & 0.0 & 0.0 & 10.5 & 2.1 & 0.0 & 17.3 & 55.2 & 0.0 & 57.0 & 33.1

Datasets:  ['casme2', 'casme3a', 'Average']
33.1 & 33.1
['casme2', 'casme3a', 'casme']


100%|█████████████████████████████████████████████| 5/5 [02:12<00:00, 26.51s/it]


MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
91.5 & 84.6 & 94.4 & 49.7 & 48.7 & 50.4 & 51.5 & 48.4 & 52.8 & 76.8 & 56.3 & 76.9 & 65.2

Datasets:  ['casme', 'casme2', 'casme3a', 'Average']
65.2 & 65.2
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
85.1 & 72.4 & 94.4 & 0.0 & 0.0 & 8.7 & 6.4 & 0.0 & 12.6 & 58.9 & 15.5 & 57.0 & 34.2

Datasets:  ['casme', 'casme2', 'casme3a', 'Average']
34.2 & 34.2
['casme2', 'casme3a', 'casme', 'samm']


...

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
92.3 & 84.1 & 95.0 & 49.7 & 48.7 & 60.6 & 57.4 & 49.4 & 67.0 & 76.6 & 60.6 & 77.3 & 68.2

Datasets:  ['casme', 'casme2', 'samm', 'casme3a', 'Average']
68.2 & 68.2
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
86.4 & 71.5 & 94.9 & 0.0 & 0.0 & 28.3 & 17.5 & 2.2 & 39.7 & 58.1 & 24.1 & 57.6 & 40.0

Datasets:  ['casme', 'casme2', 'samm', 'casme3a', 'Average']
40.0 & 40.0
['casme2', 'casme3a', 'casme', 'samm', 'mmew']


100%|█████████████████████████████████████████████| 5/5 [03:01<00:00, 36.34s/it]


MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
93.9 & 83.9 & 94.5 & 49.8 & 48.7 & 65.0 & 59.8 & 53.3 & 67.2 & 77.1 & 54.4 & 78.6 & 68.8

Datasets:  ['casme', 'casme2', 'samm', 'mmew', 'casme3a', 'Average']
68.8 & 68.8
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
89.1 & 71.1 & 94.4 & 0.0 & 0.0 & 37.0 & 22.4 & 10.2 & 40.1 & 59.4 & 11.8 & 60.1 & 41.3

Datasets:  ['casme', 'casme2', 'samm', 'mmew', 'casme3a', 'Average']
41.3 & 41.3
['casme2', 'casme3a', 'casme', 'samm', 'mmew', 'fourd']


100%|█████████████████████████████████████████████| 5/5 [03:30<00:00, 42.12s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
91.2 & 84.5 & 95.1 & 49.8 & 60.0 & 73.0 & 55.2 & 53.9 & 74.6 & 78.8 & 57.6 & 79.4 & 71.1

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
71.1 & 71.1
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
84.4 & 72.3 & 95.0 & 0.0 & 22.5 & 52.3 & 13.0 & 11.0 & 54.3 & 61.9 & 18.0 & 61.7 & 45.5

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
45.5 & 45.5


In [28]:
c3c = datasets.Casme3C(resize=64, optical_flow=True)
dfc3 = c3c.data_frame
datac3 = c3c.data

dfc3.insert(5, "apexf", dfc3["apex"] - dfc3["onset"])
dfc3.insert(6, "dataset", "casme3c")
df = pd.concat([df, dfc3])
# Fill empty action units
df = df.fillna(0)
df.loc[:, "AU1":] = df.loc[:, "AU1":].astype(int)

data = np.concatenate([data, datac3])

In [29]:
idx = df["dataset"].isin(["casme2", "casme3a", "casme3c"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [06:07<00:00, 36.78s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[[0.93257513 0.85689103 0.92772427 0.49446991 0.48697395 0.51365343
  0.49670115 0.48345389 0.59570876 0.75913929 0.48387097 0.70608339]]

Overall: 
0.6447704297758432


In [12]:
idx = df["dataset"].isin(["casme2", "casme", "casme3"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [10:15<00:00, 61.53s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.91734552 0.83766422 0.94726029 0.49715127 0.48697395 0.49453558
 0.57053988 0.48355816 0.58553502 0.75733828 0.53703871 0.72603457]

Overall: 
0.6534146206368597


In [13]:
idx = df["dataset"].isin(["casme2", "casme", "samm", "casme3"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [11:45<00:00, 70.59s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.92878457 0.82691803 0.95156128 0.49685387 0.48697395 0.58860038
 0.60231851 0.48282786 0.70119856 0.7580642  0.59482364 0.69076489]

Overall: 
0.6758074784921148


In [14]:
idx = df["dataset"].isin(["casme2", "casme", "samm", "mmew", "casme3"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [14:32<00:00, 87.23s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.93474669 0.8284188  0.94374268 0.49754613 0.48697395 0.63960877
 0.59206643 0.53074722 0.69008358 0.76896651 0.54301882 0.68936683]

Overall: 
0.6787738663626814


In [15]:
idx = df["dataset"].isin(["casme2", "casme", "samm", "mmew", "fourd", "casme3"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|██████████████████████████████████████████| 10/10 [16:44<00:00, 100.41s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.93105667 0.83672571 0.94374558 0.4979406  0.59185124 0.73182471
 0.55419973 0.54565222 0.7502311  0.77568596 0.63225343 0.70602143]

Overall: 
0.7080990318653645


## Third row
Adding modified casme3

In [37]:
use_datasets = ["casme2"]
for dataset in ["casme3a", "casme3c", "casme", "samm", "mmew", "fourd"]:
    use_datasets.append(dataset)
    idx = df["dataset"].isin(use_datasets)
    print(use_datasets)
    IValidator(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=5)

['casme2', 'casme3a']


100%|█████████████████████████████████████████████| 5/5 [01:45<00:00, 21.17s/it]


MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
93.0 & 85.6 & 92.9 & 49.4 & 48.7 & 51.3 & 49.5 & 48.4 & 55.3 & 74.5 & 48.4 & 76.9 & 64.5

Datasets:  ['casme2', 'casme3a', 'Average']
64.5 & 64.5
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
87.7 & 74.0 & 92.8 & 0.0 & 0.0 & 10.5 & 2.1 & 0.0 & 17.3 & 55.2 & 0.0 & 57.0 & 33.1

Datasets:  ['casme2', 'casme3a', 'Average']
33.1 & 33.1
['casme2', 'casme3a', 'casme3c']


100%|█████████████████████████████████████████████| 5/5 [02:11<00:00, 26.20s/it]


MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
94.1 & 87.4 & 93.0 & 49.5 & 48.7 & 50.4 & 48.5 & 48.3 & 54.9 & 76.6 & 48.4 & 61.4 & 63.4

Datasets:  ['casme2', 'casme3a', 'casme3c', 'Average']
63.4 & 63.4
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
89.5 & 77.3 & 93.1 & 0.0 & 0.0 & 8.7 & 0.0 & 0.0 & 16.5 & 58.2 & 0.0 & 27.1 & 30.9

Datasets:  ['casme2', 'casme3a', 'casme3c', 'Average']
30.9 & 30.9
['casme2', 'casme3a', 'casme3c', 'casme']


...

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
93.4 & 84.8 & 95.1 & 49.7 & 48.7 & 56.3 & 59.2 & 48.3 & 70.5 & 75.7 & 56.5 & 69.8 & 67.3

Datasets:  ['casme', 'casme2', 'samm', 'casme3a', 'casme3c', 'Average']
67.3 & 67.3
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
88.3 & 72.8 & 95.1 & 0.0 & 0.0 & 20.3 & 21.1 & 0.0 & 46.3 & 56.4 & 15.9 & 43.4 & 38.3

Datasets:  ['casme', 'casme2', 'samm', 'casme3a', 'casme3c', 'Average']
38.3 & 38.3
['casme2', 'casme3a', 'casme3c', 'casme', 'samm', 'mmew']


100%|█████████████████████████████████████████████| 5/5 [03:29<00:00, 41.82s/it]


MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
92.2 & 84.6 & 94.7 & 49.7 & 48.7 & 63.7 & 57.9 & 52.4 & 68.2 & 75.7 & 53.2 & 69.3 & 67.5

Datasets:  ['casme', 'casme2', 'samm', 'mmew', 'casme3a', 'casme3c', 'Average']
67.5 & 67.5
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
86.2 & 72.5 & 94.7 & 0.0 & 0.0 & 34.4 & 18.7 & 8.3 & 42.0 & 56.5 & 9.4 & 42.3 & 38.8

Datasets:  ['casme', 'casme2', 'samm', 'mmew', 'casme3a', 'casme3c', 'Average']
38.8 & 38.8
['casme2', 'casme3a', 'casme3c', 'casme', 'samm', 'mmew', 'fourd']


100%|█████████████████████████████████████████████| 5/5 [03:59<00:00, 47.95s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
92.2 & 84.3 & 94.3 & 49.8 & 58.0 & 70.7 & 53.7 & 54.0 & 74.4 & 78.7 & 60.5 & 67.8 & 69.9

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'casme3c', 'Average']
69.9 & 69.9
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
86.2 & 72.0 & 94.3 & 0.0 & 18.4 & 48.0 & 10.0 & 11.3 & 53.9 & 61.2 & 23.9 & 39.5 & 43.2

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'casme3c', 'Average']
43.2 & 43.2


In [47]:
# Modified casme3: only casme3a, drop samples with onset == apex, drop samples with n_frames > 16 (longer than 500ms)
idx2 = (df["dataset"] == "casme3a") #& ((df["onset"] != df["apex"]) & (df["n_frames"] < 16))

In [30]:
idx = df["dataset"].isin(["casme2", "casme3a"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [05:17<00:00, 31.71s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[[0.92832788 0.83409794 0.93396144 0.49506707 0.48697395 0.5160294
  0.49982685 0.48387097 0.53811479 0.74524549 0.48387097 0.76397523]]

Overall: 
0.642446831682417


In [48]:
idx = df["dataset"].isin(["casme2", "casme", "casme3a"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [09:03<00:00, 54.35s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.91209391 0.83127101 0.93749137 0.4970511  0.48697395 0.50905896
 0.55658349 0.4837667  0.52332092 0.76277157 0.56442964 0.77715368]

Overall: 
0.6534971900044889


In [49]:
idx = df["dataset"].isin(["casme2", "casme", "samm", "casme3a"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [10:16<00:00, 61.61s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.930481   0.8306923  0.94608392 0.49754535 0.48697395 0.63872372
 0.60518373 0.48837689 0.67737522 0.77392388 0.60391002 0.77544292]

Overall: 
0.6878927407116191


In [50]:
idx = df["dataset"].isin(["casme2", "casme", "samm", "mmew", "casme3a"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [12:55<00:00, 77.59s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.93318653 0.83463111 0.9449152  0.49744751 0.48697395 0.65835612
 0.55993225 0.53360221 0.69242073 0.77094726 0.55965273 0.78430666]

Overall: 
0.6880310215776422


In [51]:
idx = df["dataset"].isin(["casme2", "casme", "samm", "mmew", "fourd", "casme3a"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [15:04<00:00, 90.48s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.91771109 0.83000837 0.94842837 0.4979406  0.60024385 0.73651197
 0.55404105 0.54343596 0.75625474 0.78665166 0.63280343 0.8011827 ]

Overall: 
0.717101148401076


In [ ]:
!kill 11256

Testing with different modified casme3

In [40]:
idx = df["dataset"].isin(["casme2", "casme", "samm", "mmew", "fourd", "casme3"])
# Modified casme3: drop samples with onset == apex, drop samples with n_frames > 16 (longer than 500ms)
idx2 = (df["dataset"] == "casme3") & ((df["onset"] != df["apex"]) & (df["n_frames"] < 16))
idx |= idx2
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [16:33<00:00, 99.40s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.93105667 0.83672571 0.94374558 0.4979406  0.59185124 0.73182471
 0.55419973 0.54565222 0.7502311  0.77568596 0.63225343 0.70602143]

Overall: 
0.7080990318653645


In [42]:
#only part_A
idx = df["dataset"].isin(["casme2", "casme", "samm", "mmew", "fourd", "casme3a"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [15:04<00:00, 90.48s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.91771109 0.83000837 0.94842837 0.4979406  0.60024385 0.73651197
 0.55404105 0.54343596 0.75625474 0.78665166 0.63280343 0.8011827 ]

Overall: 
0.717101148401076


In [43]:
#only part_C
idx = df["dataset"].isin(["casme2", "casme", "samm", "mmew", "fourd", "casme3c"])
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [09:10<00:00, 55.09s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.92742318 0.82117555 0.95663209 0.49784198 0.65007512 0.74873082
 0.54005931 0.53880546 0.75062747 0.66610057 0.70620213 0.6994042 ]

Overall: 
0.7085898244867529


In [44]:
idx = df["dataset"].isin(["casme2", "casme", "samm", "mmew", "fourd", "casme3a"])
# Modified casme3: drop samples with onset == apex
idx2 = (df["dataset"] == "casme3a") & ((df["onset"] != df["apex"]))
idx |= idx2
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [15:05<00:00, 90.56s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.91771109 0.83000837 0.94842837 0.4979406  0.60024385 0.73651197
 0.55404105 0.54343596 0.75625474 0.78665166 0.63280343 0.8011827 ]

Overall: 
0.717101148401076


In [45]:
idx = df["dataset"].isin(["casme2", "casme", "samm", "mmew", "fourd", "casme3a"])
# Modified casme3: drop samples too long
idx2 = (df["dataset"] == "casme3a") & ((df["n_frames"] < 16))
idx |= idx2
out = IValidation(Config).validate_n_times(df[idx].reset_index(), data[idx], n_times=10)

100%|███████████████████████████████████████████| 10/10 [15:06<00:00, 90.67s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
[0.91771109 0.83000837 0.94842837 0.4979406  0.60024385 0.73651197
 0.55404105 0.54343596 0.75625474 0.78665166 0.63280343 0.8011827 ]

Overall: 
0.717101148401076
